In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn import decomposition
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import re
import string


def clean_text(s):
    s = s.split()
    s = " ".join(s)
    s = re.sub(f'[{re.escape(string.punctuation)}]', '', s)
    
    return s


In [3]:
corpus = pd.read_csv("imdb.csv", nrows=10000)
corpus.loc[:,"text"] = corpus.text.apply(clean_text)
corpus = corpus.text.values

#removing stop words
# first_sentence = (corpus[0].split())
# for words in first_sentence:
#     word = words.lower()
#     if word not in stopwords.words('english'):
#         new_corpus[0].append(words)
#     else:
#         pass


#tokenizing and vector conversion
tfv = TfidfVectorizer(tokenizer=word_tokenize, token_pattern=None)
tfv.fit(corpus)
corpus_transformed = tfv.transform(corpus)



#initialize SVD with 10 components
svd = decomposition.TruncatedSVD(n_components=10)

#fit SVD
corpus_svd = svd.fit(corpus_transformed)


sample_index = 0
feature_scores = dict(
    zip(
        tfv.get_feature_names(),
        corpus_svd.components_[sample_index]
    )
)

N=5
print(sorted(feature_scores, key=feature_scores.get, reverse=True)[:N])

['the', 'a', 'and', 'of', 'to']


In [9]:
N=5 

for sample_index in range(5):
    feature_scores = dict(
    zip(
        tfv.get_feature_names(),
        corpus_svd.components_[sample_index]
    )
)
    print(sorted(feature_scores, key=feature_scores.get, reverse=True)[:N])

['the', 'a', 'and', 'of', 'to']
['i', 'movie', 'it', 'was', 'this']
['the', 'was', 'i', 'were', 'book']
['her', 'was', 'she', 'i', 'he']
['br', 'to', 'show', 'they', 'he']


### Till now we have converted tokens into numbers and now we have to convert these integer tokens into vectors. This is known as word embeddings or word vectors
### the basic idea is to build a shallow network that learns the embeddings for words by reconstructing of an onput sentence.

### Word2Vec embedding from these two:
### predict missing words by using all other words - CBow
### take one word and predict context words - skip-gram model

### Fast-Text learns embedding for character n-grams 
### GloVe learns by co-occurence matrices

### embeddings are like dict where key is a word in corpus and value is a vector of size N

In [21]:
import numpy as np
from nltk.corpus import stopwords

def sentence_to_vec(s, embedding_dict, stop_words, tokenizer):
    words = str(s).lower()
    words = tokenizer(words)
    words = [word for word in words if word not in stop_words]
    words = [word for word in words if word.isalpha()]
    M = []
    for w in words:
        if w in embedding_dict:
            M.append(embedding_dict[w])
    if len(M) == 0:
        return np.zeros(300)
    
    M = np.array(M)
    v = M.sum(axis=0)
    return v/np.sqrt((v**2).sum())

In [22]:
sentence_to_vec(corpus[0],corpus_transformed , stopwords.words('english'), word_tokenize)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''